In [7]:
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import seaborn as sns
import statsmodels as sm
import statsmodels.api as sm
import pandas as pd
from patsy import dmatrix

C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\ledes\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ledes\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ledes\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\U

In [73]:
from geopy.geocoders import Nominatim

In [74]:
data = pd.read_csv("train.csv")

In [96]:
from geopy.geocoders import Nominatim
from urllib.request import Request

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

location = get_geolocator().reverse(a + "," + b)
address = location.address

In [104]:
pickup_lat = data["pickup_latitude"]
pickup_lon = data["pickup_longitude"]    

Columbus Circle, Upper West Side, Manhattan Community Board 4, New York County, NYC, New York, 10023, United States of America
2nd Avenue & East 24th Street, 2nd Avenue, Kips Bay, Manhattan Community Board 6, New York County, NYC, New York, 10035, United States of America
Avenue of the Americas Plaza, West 55th Street, Hell's Kitchen, Manhattan Community Board 5, New York County, NYC, New York, 10019, United States of America
377, Greenwich Street, Tribeca, Manhattan Community Board 1, New York County, NYC, New York, 10013, United States of America
Broadway & West 94th Street, Broadway, Upper West Side, Manhattan Community Board 7, New York County, NYC, New York, 10025, United States of America


In [106]:
for a, b in list(zip(pickup_lat, pickup_lon))[ : 5]  :
    location = get_geolocator().reverse(str(a) + ',' + str(b))
    address = location.address
    print(address)